In [1]:
# import required libraries
import sys
sys.path.append('../')    

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, average_precision_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import KFold

import tensorflow as tf
import tensorflow_decision_forests as tfdf

from scipy.stats import randint
import itertools

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
def process_cricket_matches(df):
    df = df.sort_values(['Match Date', 'Ground'])

    teams_a = df.iloc[::2].reset_index(drop=True)  # Even rows
    teams_b = df.iloc[1::2].reset_index(drop=True)  # Odd rows
    
    renamed_cols_a = {
        'Winner': 'Winner', 
        'Match Date': 'Match Date',
        'Ground': 'Ground',
        'Team': 'Team_A',
        'Opposition': 'Team_B',
        'RR_rolling': 'Team_A_RR_rolling',
        'batting_average_rolling': 'Team_A_batting_average_rolling',
        'BP_rolling': 'Team_A_BP_rolling',
        'SR_rolling': 'Team_A_SR_rolling',
        'bowling_average_rolling': 'Team_A_bowling_average_rolling',
        'ER_rolling': 'Team_A_ER_rolling',
        'balls_per_wicket_rolling': 'Team_A_balls_per_wicket_rolling'
    }
    
    renamed_cols_b = {
        'Team': 'Team_B_actual',  
        'Opposition': 'Team_A_actual', 
        'RR_rolling': 'Team_B_RR_rolling',
        'batting_average_rolling': 'Team_B_batting_average_rolling',
        'BP_rolling': 'Team_B_BP_rolling',
        'SR_rolling': 'Team_B_SR_rolling',
        'bowling_average_rolling': 'Team_B_bowling_average_rolling',
        'ER_rolling': 'Team_B_ER_rolling',
        'balls_per_wicket_rolling': 'Team_B_balls_per_wicket_rolling'
    }

    teams_a = teams_a.rename(columns=renamed_cols_a)
    teams_b = teams_b.rename(columns=renamed_cols_b)
    
    teams_b = teams_b[[col for col in renamed_cols_b.values()]]
    
    merged_df = pd.concat([teams_a, teams_b], axis=1)
    
    merged_df = merged_df.drop(['Team_B_actual', 'Team_A_actual'], axis=1)
    
    return merged_df

In [3]:
data = pd.read_pickle('data/matches_rolling.pkl')

In [4]:
data.head()

,Team,Opposition,Winner,Ground,Match Date,Scorecard,RR,batting_average,BP,SR,...,day_code,opp_code,target,RR_rolling,batting_average_rolling,BP_rolling,SR_rolling,bowling_average_rolling,ER_rolling,balls_per_wicket_rolling
591,Afghanistan,Ireland,Afghanistan,Dehradun,2019-03-15,Test # 2351,3.010403,38.583333,0.604752,50.173385,...,4,5,1,3.216995,11.157895,0.660377,53.616591,52.666667,4.535885,69.666667
1185,Afghanistan,Bangladesh,Afghanistan,Chattogram,2019-09-05,Test # 2361,2.906808,31.684211,0.495017,48.446805,...,3,2,1,3.086084,28.536612,0.625129,51.434730,33.867747,3.566780,54.606409
572,Afghanistan,West Indies,West Indies,Lucknow,2019-11-27,Test # 2370,2.755835,16.157895,0.534202,45.930580,...,2,10,-1,3.001292,30.025332,0.563590,50.021530,26.788449,3.235241,47.494635
52,Afghanistan,Zimbabwe,Zimbabwe,Abu Dhabi,2021-03-02,Test # 2413,2.881907,13.300000,0.548872,48.031780,...,1,11,-1,2.913343,25.056542,0.553060,48.555717,27.287702,3.317095,47.968870
50,Afghanistan,Zimbabwe,Afghanistan,Abu Dhabi,2021-03-10,Test # 2415,3.501340,93.285714,0.428790,58.355675,...,2,11,1,2.898444,19.484742,0.551075,48.307407,27.009171,3.427728,46.618700


In [5]:
data = process_cricket_matches(data)

In [6]:
data = data.drop(columns=['Scorecard', 'RR',
       'batting_average', 'BP', 'SR', 'bowling_average', 'ER',
       'balls_per_wicket', 'opp_code' , 'ground_code', 'target'])

data.head()

,Team_A,Team_B,Winner,Ground,Match Date,day_code,target,Team_A_RR_rolling,Team_A_batting_average_rolling,Team_A_BP_rolling,...,Team_A_bowling_average_rolling,Team_A_ER_rolling,Team_A_balls_per_wicket_rolling,Team_B_RR_rolling,Team_B_batting_average_rolling,Team_B_BP_rolling,Team_B_SR_rolling,Team_B_bowling_average_rolling,Team_B_ER_rolling,Team_B_balls_per_wicket_rolling
0,West Indies,England,drawn,North Sound,2015-04-13,0,0,3.039106,28.631579,0.522059,...,60.555556,3.410513,106.533333,3.727088,45.750000,0.508197,62.118126,40.312500,2.660891,90.900000
1,West Indies,England,England,St George's,2015-04-21,1,-1,2.830963,35.059928,0.528264,...,52.407629,3.584733,88.436682,3.503236,60.995997,0.485829,58.387265,36.069948,2.731188,79.528170
2,West Indies,England,West Indies,Bridgetown,2015-05-01,4,1,2.819991,33.924740,0.524637,...,60.868975,3.476495,106.535227,3.464844,55.437500,0.478016,57.747396,62.800000,3.729216,101.040000
3,Pakistan,Bangladesh,Pakistan,Mirpur,2015-05-06,2,1,3.729216,62.800000,0.426752,...,55.437500,3.464844,96.000000,3.245982,47.074694,0.506726,54.099701,32.491955,2.967049,67.708111
4,New Zealand,England,England,Lord's,2015-05-21,3,-1,3.280493,49.666667,0.389262,...,29.052632,3.163324,55.105263,3.384855,46.102029,0.517168,56.414249,33.708354,3.169700,65.577706


In [7]:
data['target'] = (data['Winner'] == data['Team_A']).astype(int)
train_df = data[data['Match Date'] <= '2021-01-01']
test_df = data[data['Match Date'] > '2021-01-01']
train_df = train_df.drop(columns='Match Date')
test_df = test_df.drop(columns='Match Date')

In [8]:
def tune_hyperparameters(df, target_col='Winner', cv_splits=5):
    param_grid = {
        'num_trees': [100, 200, 300],
        'max_depth': [20, 30, -1],
        'min_examples': [5, 10, 20],
        'bootstrap_size_ratio': [1.0, 0.8]
    }

    param_combinations = [dict(zip(param_grid.keys(), v)) 
                    for v in itertools.product(*param_grid.values())]
    
    tuning_results = []

    df = df.dropna()
    exclude_cols = [target_col, 'Match Date', 'Ground', 'Team_A', 'Team_B']
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    feature_cols = [col for col in numeric_cols if col not in exclude_cols]
    
    X = df.drop(columns=['target'])
    
    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)

    for params in param_combinations:
        print(f"\nTesting parameters: {params}")
        fold_metrics = {'precision': [], 'avg_precision': []}
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
            df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]

            train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
                df_train, label='target', task=tfdf.keras.Task.CLASSIFICATION
            )
            val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
                df_val, label='target', task=tfdf.keras.Task.CLASSIFICATION
            )
            
            model = tfdf.keras.RandomForestModel(
                num_trees=params['num_trees'],
                max_depth=params['max_depth'],
                min_examples=params['min_examples'],
                bootstrap_size_ratio=params['bootstrap_size_ratio'],
                task=tfdf.keras.Task.CLASSIFICATION
            )
            
            model.fit(train_ds, verboes=0)
            
            y_pred_proba = model.predict(val_ds)
            y_pred_binary = (y_pred_proba > 0.5).astype(int)
            
            fold_metrics['precision'].append(precision_score(df_val['target'], y_pred_binary))
            fold_metrics['avg_precision'].append(average_precision_score(df_val['target'], y_pred_proba))
        
        tuning_results.append({
            **params,
            'mean_precision': np.mean(fold_metrics['precision']),
            'std_precision': np.std(fold_metrics['precision']),
            'mean_avg_precision': np.mean(fold_metrics['avg_precision']),
            'std_avg_precision': np.std(fold_metrics['avg_precision'])
        })
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(tuning_results)
    
    # Get best parameters based on average precision
    best_params = results_df.loc[results_df['mean_avg_precision'].idxmax()]
    
    return best_params, results_df

In [9]:
best_params, tuning_results = tune_hyperparameters(train_df)


Testing parameters: {'num_trees': 100, 'max_depth': 20, 'min_examples': 5, 'bootstrap_size_ratio': 1.0}
Use /var/folders/48/k3gz2zq969q95y9yr9kffq580000gn/T/tmp7rq105rf as temporary training directory


Reading training dataset...
Training dataset read in 0:00:01.598148. Found 120 examples.
Training model...
Model trained in 0:00:00.019452
Compiling model...


I0000 00:00:1729625066.729032 6784195 kernel.cc:774] Start Yggdrasil model training
I0000 00:00:1729625066.729051 6784195 kernel.cc:775] Collect training examples
I0000 00:00:1729625066.729056 6784195 kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1729625066.729278 6784195 kernel.cc:394] Number of batches: 1
I0000 00:00:1729625066.729282 6784195 kernel.cc:395] Number of examples: 120
I0000 00:00:1729625066.729325 6784195 data_spec_inference.cc:306] 37 item(s) have been pruned (i.e. they are considered out of dictionary) for the column Ground (8 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:17296250

Model compiled.
1/1 [==============================] - 0s 29ms/step
Use /var/folders/48/k3gz2zq969q95y9yr9kffq580000gn/T/tmpoir446vz as temporary training directory


Reading training dataset...
Training dataset read in 0:00:00.133961. Found 120 examples.
Training model...
Model trained in 0:00:00.016776
Compiling model...


...

I0000 00:00:1729625155.874321 6784195 kernel.cc:774] Start Yggdrasil model training
I0000 00:00:1729625155.874334 6784195 kernel.cc:775] Collect training examples
I0000 00:00:1729625155.874338 6784195 kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1729625155.874409 6784195 kernel.cc:394] Number of batches: 1
I0000 00:00:1729625155.874414 6784195 kernel.cc:395] Number of examples: 120
I0000 00:00:1729625155.874468 6784195 data_spec_inference.cc:306] 36 item(s) have been pruned (i.e. they are considered out of dictionary) for the column Ground (8 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:17296251

Reading training dataset...
Training dataset read in 0:00:00.146558. Found 120 examples.
Training model...
Model trained in 0:00:00.017358
Compiling model...
Model compiled.
1/1 [==============================] - 0s 29ms/step
Use /var/folders/48/k3gz2zq969q95y9yr9kffq580000gn/T/tmpatniwpvc as temporary training directory


I0000 00:00:1729625156.167934 6784195 kernel.cc:774] Start Yggdrasil model training
I0000 00:00:1729625156.167947 6784195 kernel.cc:775] Collect training examples
I0000 00:00:1729625156.167954 6784195 kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1729625156.168018 6784195 kernel.cc:394] Number of batches: 1
I0000 00:00:1729625156.168032 6784195 kernel.cc:395] Number of examples: 120
I0000 00:00:1729625156.168089 6784195 data_spec_inference.cc:306] 38 item(s) have been pruned (i.e. they are considered out of dictionary) for the column Ground (7 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:17296251

Reading training dataset...
Training dataset read in 0:00:00.183287. Found 120 examples.
Training model...
Model trained in 0:00:00.016410
Compiling model...
Model compiled.
1/1 [==============================] - 0s 29ms/step


I0000 00:00:1729625156.499638 6784195 kernel.cc:774] Start Yggdrasil model training
I0000 00:00:1729625156.499650 6784195 kernel.cc:775] Collect training examples
I0000 00:00:1729625156.499654 6784195 kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1729625156.499718 6784195 kernel.cc:394] Number of batches: 1
I0000 00:00:1729625156.499723 6784195 kernel.cc:395] Number of examples: 120
I0000 00:00:1729625156.499768 6784195 data_spec_inference.cc:306] 40 item(s) have been pruned (i.e. they are considered out of dictionary) for the column Ground (6 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:17296251

In [10]:
print("\nBest Parameters (optimized for precision):")
print(best_params)


Best Parameters (optimized for precision):
num_trees               200.000000
max_depth                20.000000
min_examples              5.000000
bootstrap_size_ratio      0.800000
mean_precision            0.805817
std_precision             0.077383
mean_avg_precision        0.897798
std_avg_precision         0.064319
Name: 19, dtype: float64


In [11]:
final_model = tfdf.keras.RandomForestModel(
    num_trees=int(best_params['num_trees']),
    max_depth=int(best_params['max_depth']),
    min_examples=int(best_params['min_examples']),
    bootstrap_size_ratio=best_params['bootstrap_size_ratio'],
    task=tfdf.keras.Task.CLASSIFICATION
)

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
                train_df, label='target', task=tfdf.keras.Task.CLASSIFICATION
            )

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
                test_df, label='target', task=tfdf.keras.Task.CLASSIFICATION
            )

final_model.fit(train_ds, verbose=1)

Use /var/folders/48/k3gz2zq969q95y9yr9kffq580000gn/T/tmpzmjvf14n as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.168366. Found 238 examples.
Training model...
Model trained in 0:00:00.030043
Compiling model...
Model compiled.


I0000 00:00:1729625156.850862 6784195 kernel.cc:774] Start Yggdrasil model training
I0000 00:00:1729625156.850872 6784195 kernel.cc:775] Collect training examples
I0000 00:00:1729625156.850877 6784195 kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1729625156.850941 6784195 kernel.cc:394] Number of batches: 1
I0000 00:00:1729625156.850946 6784195 kernel.cc:395] Number of examples: 238
I0000 00:00:1729625156.851020 6784195 data_spec_inference.cc:306] 43 item(s) have been pruned (i.e. they are considered out of dictionary) for the column Ground (26 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:1729625

In [12]:
y_pred_proba = final_model.predict(test_ds)
y_pred_binary = (y_pred_proba > 0.5).astype(int)

precision_score(y_pred_binary, test_df['target'])

1/1 [==============================] - 0s 35ms/step


0.8205128205128205

In [13]:
combined = pd.DataFrame(dict(actual=test_df['target'], prediction=y_pred_binary.flatten()))
pd.crosstab(index=combined['actual'], columns=combined['prediction'])

prediction,0,1
actual,,
0,65,6
1,14,64
